<h2>Importing Libraries</h2>
<h6>
    <ul>
        <li><a href="https://selenium-python.readthedocs.io/index.html">Selenium (pip install selenium)</a></li>
        <li><a href="https://pypi.org/project/webdriver-manager/">Webdriver Manager (pip install webdriver-manager)</a></li>
        <li><a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/">BeautifulSoup (pip install beautifulsoup4)</a></li>
        <li><a href="https://docs.python.org/3/library/time.html">Time</a> [NATIVE LIBRARY]
        <li><a href="https://docs.python.org/3/library/datetime.html">Datetime</a> [NATIVE LIBRARY]
        <li><a href="https://docs.python.org/3/library/csv.html">CSV</a> [NATIVE LIBRARY]
        <li><a href="https://pandas.pydata.org/docs/">Pandas (pip install pandas)</a> [OPTIONAL]</li>
        <li><a href="https://pypi.org/project/requests/">Requests (python -m pip install requests)</a> [UNUSED]</li>        
</h6>


In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import datetime
import csv
import pandas as pd
#import requests

<h2>Constants</h2>


In [2]:
BASE_URL='https://br.indeed.com/'
HEADERS ={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

<h2>Functions</h2>

<h3>Set Up</h3>

In [3]:
def set_up():
    what=input('O quê: ')
    what=what.replace(' ','+')
    where=input('Onde: ')
    where=where.replace(' ','+')
    range_=int(input('Alcance: '))
    return what, where, range_

<h3>Scraping</h3>

In [4]:
def build_url(BASE_URL,what,where,page=None):
    url=f'{BASE_URL}jobs?q={what}&l={where}&start={page}'
    return url

In [5]:
def get_page_content(driver, url):
    #driver=webdriver.Firefox()
    time.sleep(1)
    driver.get(url)
    content=driver.page_source
    return content

In [6]:
#============================Unused============================#
def get_jobs(driver,content):
    soup=BeautifulSoup(content,'lxml')
    jobs_list=[]   
    for post in soup.select('.job_seen_beacon'):
        try:
            data={
                "Vaga":post.select('.jobTitle')[0].get_text().strip(),
                "Empresa":post.select('.companyName')[0].get_text().strip(),
                "Avaliação":post.select('.ratingNumber')[0].get_text().strip(),
                "Local":post.select('.companyLocation')[0].get_text().strip(),
                "Data":post.select('.date')[0].get_text().strip(),
                "Descrição":post.select('.job-snippet')[0].get_text().strip(),
                #"Link":f'br.indeed.com'+post.select('.jcs-JobTitle css-jspxzf eu4oa1w0').get('href').strip()
                "Link":f'br.indeed.com'+post.find('a',{'class':'jcs-JobTitle css-jspxzf eu4oa1w0'}).get('href').strip()    
                }
        except IndexError:
            continue
        jobs_list.append(data)
    #dataframe = pd.DataFrame(jobs_list)
    
    return jobs_list

In [7]:
#============================Unused============================#
def get_current_url(driver, url, what, where=None):
    #driver = webdriver.Firefox()
    #driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(1)
    driver.find_element(By.XPATH,'//*[@id="text-input-what"]').send_keys(what)
    time.sleep(1)
    if where is not None:
        driver.find_element(By.XPATH,'//*[@id="text-input-where"]').send_keys(where)
        time.sleep(1)
    driver.find_element(By.XPATH,'/html/body/div').click()
    time.sleep(1)
    try:
        driver.find_element(By.XPATH,'//*[@id="jobsearch"]/button').click()
    except:
        driver.find_element(By.XPATH,'//*[@id="whatWhereFormId"]/div[3]/button').click()
    current_url = driver.current_url

    return current_url

In [8]:
def scrape(driver,what,where,range_):
    jobs=[]
    for page in range(0,(range_*10),10):
        url=build_url(BASE_URL,what,where,page)
        content=get_page_content(driver,url)
        soup=BeautifulSoup(content,'lxml')   
        for post in soup.select('.job_seen_beacon'):
            try:
                data={
                    "Vaga":post.select('.jobTitle')[0].get_text().strip(),
                    "Empresa":post.select('.companyName')[0].get_text().strip(),
                    "Avaliação":post.select('.ratingNumber')[0].get_text().strip(),
                    "Local":post.select('.companyLocation')[0].get_text().strip(),
                    "Data":post.select('.date')[0].get_text().strip(),
                    "Descrição":post.select('.job-snippet')[0].get_text().strip(),
                    #"Link":f'br.indeed.com'+post.select('.jcs-JobTitle css-jspxzf eu4oa1w0').get('href').strip()
                    "Link":f'br.indeed.com'+post.find('a',{'class':'jcs-JobTitle css-jspxzf eu4oa1w0'}).get('href').strip()    
                }
            except IndexError:
                continue
            jobs.append(data)
    return jobs    

<h3>CSV Export</h3>

In [9]:
def save_as_csv(jobs):
    file=open('jobs.csv',"w",encoding="utf-8")
    writer=csv.writer(file)
    writer.writerow(['Vaga','Empresa','Avaliação','Local','Data','Descrição','Link'])
    for job in jobs:
        writer.writerow(list(job.values()))

<h1>Execution</h1>

<h3>Default Execution</h3>

In [10]:
what,where,range_=set_up()
driver = webdriver.Firefox()
jobs=scrape(driver,what,where,range_)
driver.quit()
#jobs
save_as_csv(jobs)

<h3>Pandas Datasheet</h3>

In [19]:
dataframe=pd.DataFrame(jobs)
now=datetime.datetime.now()
formatted_date,formatted_time=now.strftime("%d/%m/%Y"),now.strftime("%H:%M:%S %p")
what,where=what.replace('+',' '),where.replace('+',' ')
print(f'O quê: {what} | Onde: {where} | Alcance: {range_} | De: {formatted_date} - {formatted_time}')
dataframe

O quê: Python | Onde: Rio de Janeiro,RJ | Alcance: 10 | De: 03/05/2023 - 13:34:40 PM


,Vaga,Empresa,Avaliação,Local,Data,Descrição,Link
0,Analista de Dados,Morning Star Consulting,"4,3","Rio de Janeiro, RJ",EmployerAtiva 7 dias atrás,"Conhecimento sólido em Power BI, Python, Spotf...",br.indeed.com/company/Morning-Star-Consulting/...
1,Analista de Inteligência de Negócios,Grupo Trigo,"4,4","Rio de Janeiro, RJ",PostedAnunciada há 5 dia,Apoio na criação de painéis no Power BI;\nNece...,br.indeed.com/rc/clk?jk=c8d6335e5abac31c&fccid...
2,Estágio em Análise de Dados e Performance - Ri...,Baker Hughes,"4,0","Rio de Janeiro, RJ",PostedAnunciada há 7 dia,"Atuar na concepção, arquitetura e desenvolvime...",br.indeed.com/rc/clk?jk=d7201af21e603927&fccid...
3,Analista de BI JR,Vibra Energia,"3,7","Rio de Janeiro, RJ",PostedAnunciada há 15 dia,Experiência desejavel com programação em Pytho...,br.indeed.com/rc/clk?jk=9d49536bba430e0a&fccid...
4,ANL CUSTOMER SUCCESS I,Energisa - Recrutamento Externo,"5,0","Rio de Janeiro, RJ",PostedAnunciada há 19 dia,Monitorar performance da base de clientes das ...,br.indeed.com/rc/clk?jk=49e891ba9fb983c7&fccid...
5,ANALISTA DE NEGÓCIOS (BI),DROGARIA VENANCIO,"3,7","Rio de Janeiro, RJ",PostedAnunciada há 15 dia,"A Drogaria Venâncio, uma das maiores redes de ...",br.indeed.com/rc/clk?jk=4b2dfe09d3e5c493&fccid...
6,DATA SCIENCE CONSULTANT RIO DE JANEIRO,managementsolutions,"3,6","Rio de Janeiro, RJ",PostedAnunciada há mais de 30 dias,You will be working in key projects for leadin...,br.indeed.com/rc/clk?jk=c2d10c8b10445f26&fccid...
7,ANALISTA DE POWER BI,Vikservices,"3,9","Rio de Janeiro, RJ",PostedHoje,Desejável/Diferencial: Excel avançado Power po...,br.indeed.com/rc/clk?jk=9cf163823cd9d7f1&fccid...
8,Desenvolvedor Full Stack,Curriculum.com.br,"4,1","Rio de Janeiro, RJ",PostedAnunciada há 10 dia,"Requisitos desejáveis BigData, Azure Máquina V...",br.indeed.com/rc/clk?jk=8ed6ca9424d99d25&fccid...
9,Analista de Sistemas Pleno,CONQUEST ONE,"3,8","Home office in Rio de Janeiro, RJ",PostedAnunciada há mais de 30 dias,Descrição A Conquest One é uma empresa especia...,br.indeed.com/rc/clk?jk=1c17e5c03bcd930a&fccid...
